

# <center>  Aprendizaje No Supervisado: Clustering </center>

## Descripción
En esta lección se abordan los algoritmos de clusterización: K-Means e Hierarchical clustering

## Contenido
* Importación de librerias y módulos
* Carga dataset de trabajo
* Análisis exploratorio de datos
* Clusterización

## Requisitos previos

* Haber completado los cursos:
  - Introducción a Python
  - Estadística para Ciencia de Datos
  - Introducción a Machine Learning


### Importar librerías y verificar versiones

In [0]:
import sys
import datetime as dt
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib
import seaborn as sns

print('Python:', sys.version)
print('NumPy:', np.__version__)
print('Pandas:', pd.__version__)
print('Seaborn:', sns.__version__)
print('Matplotlib:', matplotlib.__version__)
print('Scikit-learn:', sk.__version__)

## 1. Importar módulos específicos de librerías

In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
import matplotlib.pyplot as plt

## 2. Dataset de Trabajo

**df_retail:** Este conjunto de datos contiene todas las transacciones que ocurren para un minorista en línea (online) registrado en el Reino Unido entre el 2010-12-01 y el 2011-12-09. La compañía vende principalmente artículos de regalo únicos para toda ocasión. Muchos clientes de la empresa son mayoristas.

<a href='https://storage.googleapis.com/datasets-academy/Track%20Data%20Science/02%20-%20Introduccion%20a%20Machine%20Learning/OnlineRetail.csv'>
  Link para descargar el dataset OnlineRetail</a>

**Objetivo del Clustering:** Segmentar a los clientes de la tienda online de acuerdo a los criterios de recencia, frecuencia y monto (RFM), para que la empresa pueda identificar a los clientes en base a estas características y realizar una campaña de marketing focalizada

**Diccionario de datos**

1. InvoiceNo: número de factura. Tipo nominal. Un número integral de 6 dígitos asignado exclusivamente a cada transacción. Si este código comienza con la letra 'c', indica una cancelación.
2. StockCode: código de producto (artículo). Tipo Nominal. Un número integral de 5 dígitos asignado exclusivamente a cada producto distinto.
3. Description: Nombre del producto (artículo). Tipo Nominal.
4. Quantity: las cantidades de cada producto (artículo) por transacción. Tipo Numérico.
5. InvoiceDate: fecha y hora de inicio. Tipo Date. El día y la hora en que se generó una transacción.
6. UnitPrice: precio unitario. Tipo Numérico. Precio del producto por unidad en libras esterlinas (£).
7. CustomerID: número de cliente. Tipo Nominal. Un número integral de 5 dígitos asignado exclusivamente a cada cliente.
8. Country: nombre del país. Tipo Nominal. El nombre del país donde reside el cliente.

In [0]:
ruta='https://storage.googleapis.com/datasets-academy/Track%20Data%20Science/02%20-%20Introduccion%20a%20Machine%20Learning/OnlineRetail.csv'

In [0]:
# Carga dataset
df_retail = pd.read_csv(ruta, encoding = 'ISO-8859-1', header = 0)
df_retail.head()

In [0]:
# Examinar número de filas y columnas
df_retail.shape

In [0]:
# Analizar el tipo de datos de cada variable
df_retail.info()

Se comprueba que algunas columnas poseen un tipo incorrecto conforme a nuestro diccionario de datos ([revisa aqui los formatos para fechas](https://docs.python.org/2.6/library/datetime.html#strftime-strptime-behavior))

In [0]:
# Transformar a columnas a formatos correctos de acuerdo al diccionario de datos
df_retail['CustomerID'] = df_retail['CustomerID'].astype(str)
df_retail['InvoiceDate'] = pd.to_datetime(df_retail['InvoiceDate'], format = '%d-%m-%Y %H:%M')
df_retail.head()

In [0]:
df_retail.dtypes

## 3. Análisis exploratorio de datos

In [0]:
df_retail.describe(include = np.number)

Al revisar la calidad de los datos en las variables `Quantity` y `UnitPrice`, se evidencian valores negativos. Por conocimiento del negocio se sabe que si el `InvoceNo` comienza por la letra `C` significa que fue una cancelación.

In [0]:
df_retail.loc[df_retail['Quantity'] <= 0].head(10)

In [0]:
df_retail.loc[df_retail['UnitPrice'] <= 0].head(10)

Se procede a filtar aquellas filas en ambas columnas que poseen valores mayores a cero

In [0]:
df_retail_fitrado = df_retail.loc[(df_retail['Quantity'] > 0) & (df_retail['UnitPrice'] > 0)].copy()
df_retail_fitrado.shape

Calcular la proporción de valores faltantes por columna

In [0]:
df_retail_fitrado.isnull().sum() / len(df_retail_fitrado)

### 3.1 Transformaciones para el análisis RFM

In [0]:
# Calcular el monto de las ventas (precio * cantidad)
df_retail_fitrado['Total_Sales'] = df_retail_fitrado['Quantity'] * df_retail_fitrado['UnitPrice']
df_retail_fitrado.head()

In [0]:
# Cálculo para la recencia: Guardamos la fecha de la última transacción
fecha_max = df_retail_fitrado['InvoiceDate'].max()
fecha_max

In [0]:
# Calcular la diferencia entre la fecha de la última transacción y la fecha que se realizó la transacción
df_retail_fitrado['Date_difference'] = fecha_max - df_retail_fitrado['InvoiceDate']

#Convertir en número de días usando datetime
df_retail_fitrado['Date_difference'] = df_retail_fitrado['Date_difference'].dt.days
df_retail_fitrado.head()

Procedemos a realizar el agrupamiento por cliente de acuerdo a la columna CustomerID siguiendo la siguiente lógica para el análisis de RFM:
- Monto: Sumatoria de las ventas totales
- Frecuencia: Conteo de la cantidad de productos vendidos
- Recencia: Mínimo número de días desde que realizó una transacción con respecto a la fecha de corte

In [0]:
# Agrupar clientes por Monto, Frecuencia y Recencia
df_rfm = df_retail_fitrado.groupby('CustomerID').agg({'Total_Sales': 'sum', 'Quantity': 'count', 'Date_difference': 'min'})
df_rfm.head()

In [0]:
df_rfm.shape

In [0]:
# Renombrar columnas
df_rfm.rename(columns = {'Total_Sales': 'Monto', 'Quantity': 'Frecuencia', 'Date_difference': 'Recencia'}, inplace = True)
df_rfm.columns

### 3.2 Análisis de valores atípicos

Aplicar la visualización box-plot para cada columna del dataframe:

In [0]:
display(df_rfm)

In [0]:
# Remover los outliers para la Recencia
Q1 = df_rfm.Recencia.quantile(0.25)
Q3 = df_rfm.Recencia.quantile(0.75)
IQR = Q3 - Q1
df_limpio_rec = df_rfm[(df_rfm.Recencia >= Q1 - 1.5*IQR) & (df_rfm.Recencia <= Q3 + 1.5*IQR)].copy()

In [0]:
# Remover los outliers para la Frecuencia
Q1 = df_limpio_rec.Frecuencia.quantile(0.25)
Q3 = df_limpio_rec.Frecuencia.quantile(0.75)
IQR = Q3 - Q1
df_limpio_frec = df_limpio_rec[(df_limpio_rec.Frecuencia >= Q1 - 1.5*IQR) & (df_limpio_rec.Frecuencia <= Q3 + 1.5*IQR)].copy()

In [0]:
# Remover los outliers para la Monto
Q1 = df_limpio_frec.Monto.quantile(0.25)
Q3 = df_limpio_frec.Monto.quantile(0.75)
IQR = Q3 - Q1
df_limpio_final = df_limpio_frec[(df_limpio_frec.Monto >= Q1 - 1.5*IQR) & (df_limpio_frec.Monto <= Q3 + 1.5*IQR)].copy()

In [0]:
df_limpio_final.shape

### 3.3  Visualización descriptiva del DataFrame

In [0]:
sns.pairplot(df_limpio_final)

### 3.4 Escalamiento de los datos

In [0]:
rfm_escalado = StandardScaler().fit_transform(df_limpio_final)
rfm_escalado

In [0]:
type(rfm_escalado)

## 4. Clusterización

Se analizará dos algoritmos para realizar el proceso de agrupamientos de las observaciones

### 4. K-Means Clustering

Usando Scikit-learn, debemos primero importar la instancia de ```sklearn.cluster.KMeans```, e invocar los métodos necesarios:

1. Preparar los datos X
2. Crear instancia de ```sklearn.cluster.KMeans```:

``` python
km = KMeans(n_clusters = 3, 
            init       = "k-means++",
            n_jobs     = 4, 
            random_state = 0
           )
```
3. Entrenar:
```
km.fit(X)
```
4. Identificar etiquetas de clasificación:
```
km.labels_
```

A continuación describiremos los argumentos más comunes:


---
**```sklearn.cluster.KMeans(n_clusters=8, init=’k-means++’, n_init=10, max_iter=300, tol=0.0001, precompute_distances=’auto’, verbose=0, random_state=None, copy_x=True, n_jobs=None, algorithm=’auto’)[source]```**

[Documentación](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans)

In [0]:
# Para encontrar el número óptimo de clusters
wcss = []

for i in range(1, 11):
    kmeans = KMeans(n_clusters   = i, 
                    init         = 'k-means++', #selecciona los centroides de los clústeres iniciales de una manera inteligente para acelerar la convergencia
                    max_iter     = 300, 
                    n_init       = 10, 
                    random_state = 0)
    kmeans.fit(rfm_escalado)
    wcss.append(kmeans.inertia_)

In [0]:
#Crear un dataframe con la métrica de evaluación y el número de clústeres
numero_clusteres = list(enumerate(wcss, start = 1))
metrica_wcss = np.array(numero_clusteres)
df_wcss= pd.DataFrame(data = metrica_wcss, columns = ['CLUSTERS','WCSS'], index = range(1, 11))
df_wcss

El número óptimo de clusters está dado por el lugar de ocurrencia del "codo" (elbow), evaluando un gráfico de línea:

In [0]:
display(df_wcss)

Aplicamos KMeans con el número de clústers óptimo

In [0]:
kmeans = KMeans(n_clusters = 2, 
                init = 'k-means++', 
                max_iter = 300, 
                n_init = 10, 
                random_state = 0)

kmeans.fit(rfm_escalado)

In [0]:
# Inercia: Suma de los cuadrados de las distancias de las observaciones respecto de sus centros
kmeans.inertia_

In [0]:
# Guardamos las etiquetas de cada cliente (cada clúster)
etiquetas = kmeans.labels_
etiquetas

**Visualización por cluster**

In [0]:
# Creación de una nueva columna en el dataframe con las etiquetas
df_limpio_final['Cluster_kmeans'] = etiquetas
df_limpio_final.head()

In [0]:
# Grafico de dispersión
display(df_limpio_final)

In [0]:
df_limpio_final.groupby('Cluster_kmeans').agg('mean')

### 4.2 Hierarchical Clustering

La familia de clústers jerárquicos, agrupa las observaciones de forma iterativa hasta alcanzar un solo cluster final. El usuario es el encargado de seleccionar el número de clusters deseados al final.

En principio, cada observación es en si misma un cluster. En una segunda etapa, se agrupan las observaciones más cercanas en función de un criterio de agrupamiento de la siguiente lista:


* **Ward** minimiza la suma de cuadrados de la diferencias dentro de los clusters. Es un enfoque de minimización de la varianza y, en ese sentido, actua de forma similar a K-Means.
* **Maximum or complete linkage** minimiza la distancia máxima entre un par de clusters.
* **Average linkage** minimiza el promedio de las distancias entre todas las observaciones de un par de clusters.
* **Single linkage** minimiza la distancia entre las observaciones más cercanas de un par de clusters.



Usando Scikit-learn, debemos primero importar la instancia de ```sklearn.cluster.AgglomerativeClustering```, e invocar los métodos necesarios:

1. Preparar los datos X
2. Crear instancia de ```sklearn.cluster.AgglomerativeClustering```:

``` python
hc = AgglomerativeClustering(linkage    = "ward", 
                             affinity   = "euclidean"
                             n_clusters = 3
                            )
```
3. Entrenar:
```
hc.fit(X)
```
4. Identificar etiquetas de clasificación:
```
hc.labels_
```

A continuación describiremos los argumentos más comunes:


---
```
sklearn.cluster.AgglomerativeClustering(n_clusters=2, affinity=’euclidean’, memory=None, 
connectivity=None, compute_full_tree=’auto’, linkage=’ward’, pooling_func=’deprecated’, distance_threshold=None)
```

[Documentación](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html#sklearn.cluster.AgglomerativeClustering)

**Visualización dendogramas**

In [0]:
ward_euclidian = linkage(rfm_escalado, method="ward", metric='euclidean')

plt.figure(figsize=(22,8))
dendrogram(ward_euclidian)
plt.show()

#### Aplicación clusterización jerárquica

In [0]:
hc = AgglomerativeClustering(linkage    = "ward", 
                             affinity   = "euclidean",
                             n_clusters = 2)
hc.fit(rfm_escalado)

In [0]:
etiquetas_hc = hc.labels_
etiquetas_hc

**Visualización por cluster**

In [0]:
# Creación de una nueva columna en el dataframe con las etiquetas
df_limpio_final['Cluster_hc'] = etiquetas_hc
df_limpio_final.head()

In [0]:
# Gráfico de dispersion
display(df_limpio_final)

>**Ejercicio:** 

1. Aplicar la clusterización KMeans de los datos RFM estandarizados `rfm_escalado`. Determinando la existencia de 3 clústers, obtenga las etiquetas correspondientes para cada cliente y realice las visualizaciones para interpretar cada clúster. Parámetros: `n_clusters = 3, init=’k-means++’, n_init=10, max_iter=300, random_state=0`

2. Aplicar la clusterización Hierarchical Clustering de los datos RFM estandarizados `rfm_escalado`. Empleando la distancia euclídea y la forma de cálculo (linkage) sea `complete` realice la clusterización jerárquica con 4 clústers. Por último, realice las visualizaciones para interpretar cada clúster. Parámetros = `linkage = "complete", affinity = "euclidean", n_clusters = 4`

In [0]:
# Su codigo aquí

In [0]:
#Kmean
kmeans = KMeans(n_clusters = 3, 
                init = 'k-means++', 
                max_iter = 300, 
                n_init = 10, 
                random_state = 0)
kmeans.fit(rfm_escalado)
df_limpio_final['km_labels_3'] = kmeans.labels_
display(df_limpio_final)

Monto,Frecuencia,Recencia,Cluster_kmeans,Cluster_hc,km_labels_3,hc_labels_4
1797.24,31,74,1,1,1,2
1757.55,73,18,1,1,1,1
334.40000000000003,17,309,0,0,0,0
2506.040000000001,85,35,1,1,1,2
89.0,4,203,0,0,0,0
1079.4,58,231,0,0,0,0
459.4,13,213,0,0,0,0
2811.4300000000007,59,22,1,1,1,2
1168.06,19,1,0,1,2,1
2662.0600000000004,129,51,1,1,1,2


In [0]:
#HC
hc = AgglomerativeClustering(linkage    = "complete", 
                             affinity   = "euclidean",
                             n_clusters = 4)
hc.fit(rfm_escalado)
df_limpio_final['hc_labels_4'] = hc.labels_
display(df_limpio_final)

Monto,Frecuencia,Recencia,Cluster_kmeans,Cluster_hc,km_labels_3,hc_labels_4
1797.24,31,74,1,1,1,2
1757.55,73,18,1,1,1,1
334.40000000000003,17,309,0,0,0,0
2506.040000000001,85,35,1,1,1,2
89.0,4,203,0,0,0,0
1079.4,58,231,0,0,0,0
459.4,13,213,0,0,0,0
2811.4300000000007,59,22,1,1,1,2
1168.06,19,1,0,1,2,1
2662.0600000000004,129,51,1,1,1,2
